In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set()

In [ ]:
raw_ds = pd.read_csv('high_diamond_ranked_10min.csv')
raw_ds.head()

In [ ]:
raw_ds.info()

In [ ]:
raw_ds.shape

In [ ]:
#drop id column
df = raw_ds.drop('gameId', axis=1, inplace = False)
df.head()

In [ ]:
# Seperate target column
target = df['blueWins']
df.drop('blueWins', axis=1, inplace=True)

In [ ]:
# Split data into TTS
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.3, random_state =41)

In [ ]:
# Feature Selection 
# Information Gain

from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt 
%matplotlib inline

plt.figure(figsize=(20,10))
importances = mutual_info_regression(df, target)
feat_importances = pd.Series(importances, df.columns)
feat_importances.plot(kind='barh', color='teal')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
cor = X_train.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
def correlation(data,threshold):
    col_corr = set()
    corr_matrix = data.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i,j] > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
correlated_features = correlation(X_train, 0.8)

In [ ]:
X_train.drop(correlated_features, axis=1, inplace=True)
X_test.drop(correlated_features, axis=1, inplace=True)

In [ ]:
# ANN Model
import tensorflow as tf

inputs = tf.keras.Input(X_train.shape[1])
x = tf.keras.layers.Dense(32, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(16, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Dense(units = 16, activation='relu',input_dim=X_train.shape[1]))
# model.add(tf.keras.layers.Dense(units = 16, activation='relu',input_dim=16))
# model.add(tf.keras.layers.Dense(units=2, activation='softmax'))
# model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
import numpy as np
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
model.fit(X_train, y_train, epochs=10)

In [ ]:
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()


In [71]:
results = model.evaluate(X_test, y_test, batch_size=128)

2964/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================